<a href="https://colab.research.google.com/github/iljf/NLU_project_team1/blob/main/KLUE_RoBERTa%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np 
import torch
import random
import tarfile

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CosineSimilarity, MSELoss
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [4]:
# Klue dataset 다운
!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

--2022-05-25 11:56:28--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.218.240.130
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.218.240.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz.1’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  1.36MB/s    in 0.9s    

2022-05-25 11:56:30 (1.36 MB/s) - ‘klue-sts-v1.1.tar.gz.1’ saved [1349881/1349881]



In [5]:
tar_file = tarfile.open('/content/drive/MyDrive/NLP/klue-sts-v1.1.tar.gz')
tar_file.extractall(path='/content/drive/MyDrive/NLP')
tar_file.close()

In [6]:
df1 = pd.read_json('/content/drive/MyDrive/NLP/klue-sts-v1.1/klue-sts-v1.1_train.json')
test = pd.read_json('/content/drive/MyDrive/NLP/klue-sts-v1.1/klue-sts-v1.1_dev.json')

In [7]:
df1.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [8]:
df1.shape

(11668, 6)

In [9]:
# 중복문장 확인
df1.duplicated(['sentence1', 'sentence2']).sum()

7

In [10]:
# 결측치 확인
df1.isnull().sum()

guid           0
source         0
sentence1      0
sentence2      0
labels         0
annotations    0
dtype: int64

In [11]:
# 중복문장 제거
df1 = df1.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [12]:
# 3종류의 라벨링 binaly 1이 같은문장 2가 다른문장 
df1.labels[0]

{'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714}

In [13]:
df1.labels[2]

{'binary-label': 0,
 'label': 0.30000000000000004,
 'real-label': 0.33333333333333304}

In [14]:
# 라벨 분리
labels = df1.labels.to_list()
labels = pd.DataFrame(labels)
print(len(labels))
labels

11661


,label,real-label,binary-label
0,3.7,3.714286,1
1,0.0,0.000000,0
2,0.3,0.333333,0
3,0.6,0.571429,0
4,4.7,4.714286,1
...,...,...,...
11656,4.0,4.000000,1
11657,0.0,0.000000,0
11658,3.7,3.666667,1
11659,4.7,4.714286,1


In [15]:
labels['label'].value_counts()
la1 = len(labels.loc[(labels['label'] >= 0) & (labels['real-label'] < 1.0)])
la2 = len(labels.loc[(labels['label'] >= 1.0) & (labels['real-label'] < 2.0)])
la3 = len(labels.loc[(labels['label'] >= 2.0) & (labels['real-label'] < 3.0)])
la4 = len(labels.loc[(labels['label'] >= 3.0) & (labels['real-label'] < 4.0)])
la5 = len(labels.loc[(labels['label'] >= 4.0) & (labels['real-label'] < 5.0)])

print(f' Score 1미만인 데이터: {la1}\n Score 2미만인 데이터: {la2}\n Score 3미만인 데이터: {la3}\n Score 4미만인 데이터: {la4}\n Score 5미만인 데이터: {la5}')  

 Score 1미만인 데이터: 4350
 Score 2미만인 데이터: 906
 Score 3미만인 데이터: 810
 Score 4미만인 데이터: 2852
 Score 5미만인 데이터: 2698


In [16]:
# binary-labele 은 score 3미만인 데이터로 이루어져 있다.
labels['binary-label'].value_counts()

0    6066
1    5595
Name: binary-label, dtype: int64

In [17]:
df1_1 = df1[['sentence1', 'sentence2']].join(labels['binary-label'])
df1_1.head()

,sentence1,sentence2,binary-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1


In [18]:
# KorNLUdataset 다운
!git clone https://github.com/kakaobrain/KorNLUDatasets/

fatal: destination path 'KorNLUDatasets' already exists and is not an empty directory.


In [19]:
# Question pair 다운
!git clone https://github.com/songys/Question_pair

fatal: destination path 'Question_pair' already exists and is not an empty directory.


In [20]:
path = '/content/drive/MyDrive/NLP/KorNLUDatasets/KorSTS/sts-train.tsv'

In [21]:
df2 = pd.read_csv(path, sep="\t+")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [22]:
# 스코어로 되어있는 형태
df2

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.
...,...,...,...,...,...,...,...
5744,main-news,headlines,2016,1456,0.00,폭풍우 클로다흐가 영국을 강타하면서 심한 강풍,메르켈은 나토와 라트비아의 연대를 약속한다.
5745,main-news,headlines,2016,1465,0.00,리비아 테러리스트들이 공습에 대한 복수로 찍은 수십 명의 이집트 인질들,나일강에서 더 많은 시체가 발견되면서 이집트 보트 충돌 사고 사망자 수가 증가한다.
5746,main-news,headlines,2016,1466,0.00,바레인으로 향하는 대통령,시 주석 : 에볼라 퇴치를 계속 돕기 위한 중국
5747,main-news,headlines,2016,1470,0.00,"중국, 인도는 양국 관계를 증진시키겠다고 맹세한다",중국은 불안한 주식 거래자들을 안심시키기 위해 뒤뚱거리고 있다.


In [23]:
# 중복값 확인
df2.duplicated(['sentence1', 'sentence2']).sum()

49

In [24]:
# 중복값 제거
df2 = df2.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [25]:
# 결측치 확인
df2.isnull().sum()

genre        0
filename     0
year         0
id           0
score        0
sentence1    0
sentence2    0
dtype: int64

In [26]:
# 위와 같이 스코어 3 미만은 0 3 이상은 1 로 분류
df2['binary-label'] = df2['score'].apply(lambda x: 0 if x < 3  else  1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
# 스코어 확인
la11 = len(df2.loc[(df2['score'] >= 0) & (df2['score'] < 3.0)])
la11

2729

In [28]:
# 분류 확인
df2['binary-label'].value_counts()

1    2971
0    2729
Name: binary-label, dtype: int64

In [29]:
df2_1 = df2[['sentence1', 'sentence2', 'binary-label']]
df2_1.head()

,sentence1,sentence2,binary-label
0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,1
1,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,1
2,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,1
3,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.,0
4,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.,1


In [30]:
path2 = '/content/drive/MyDrive/NLP/Question_pair/kor_pair_train.csv'

In [31]:
df3 = pd.read_csv(path2)

In [32]:
# 1이 다른문장 2가 같은문장
df3

,id,qid1,qid2,question1,question2,is_duplicate
0,1,1,2,1000일 만난 여자친구와 이별,10년 연예의끝,1
1,2,3,4,10년 연애사 되돌아보니 다 부질없네,10년이라는 시간이 참 무색하다 싶네,1
2,3,5,6,10년만나다 헤어지네,14년된 여자친구랑 이별,1
3,4,7,8,10월의 마지막밤 . 더 보고싶네 그사람,15년…안녕,1
4,5,9,10,14년의 기나긴 이야기.,1년 9개월 의 연애 종지부.,1
...,...,...,...,...,...,...
6883,6884,13767,13768,O형 남자와 썸을 타는데,O형 여자와 썸을 타는데,1
6884,6885,13769,13770,SD카드 안돼,SD카드 망가졌어,0
6885,6886,13771,13772,SNS 를 끊어야 하는데.,가슴 아픈 이별,1
6886,6887,13773,13774,SNS 시간낭비인데 자꾸 보게됨,SNS 시간낭비인 거 아는데 매일 하는 중,0


In [33]:
# 중복값 확인
df3.duplicated(['question1', 'question2']).sum()

1

In [34]:
# 중복값 제거
df3 = df3.drop_duplicates(['question1', 'question2'], keep='first', ignore_index=True)

In [35]:
# 결측치 확인
df3.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [36]:
# 분류 기준 바꾸기
df3['binary-label'] = df3['is_duplicate'].apply(lambda x: 0 if x > 0 else 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
# 분류 확인
df3['is_duplicate'].value_counts()

0    4749
1    2138
Name: is_duplicate, dtype: int64

In [38]:
# 분류 확인
df3['binary-label'].value_counts()

1    4749
0    2138
Name: binary-label, dtype: int64

In [39]:
df3_1 = df3[['question1', 'question2', 'binary-label']]

In [40]:
df3_1.rename(columns = {'question1':'sentence1','question2':'sentence2'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
df = pd.concat([df1_1,df2_1, df3_1],axis=0, join='inner', ignore_index=True) 

In [42]:
df

,sentence1,sentence2,binary-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1
...,...,...,...
24243,O형 남자와 썸을 타는데,O형 여자와 썸을 타는데,0
24244,SD카드 안돼,SD카드 망가졌어,1
24245,SNS 를 끊어야 하는데.,가슴 아픈 이별,0
24246,SNS 시간낭비인데 자꾸 보게됨,SNS 시간낭비인 거 아는데 매일 하는 중,1


In [43]:
def cleansing(text):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [44]:
df['clean1'] = df['sentence1'].apply(cleansing)
df['clean2'] = df['sentence2'].apply(cleansing)

In [45]:
df= df[['clean1', 'clean2', 'binary-label']]

In [46]:
df.rename(columns = {'clean1':'sentence1','clean2':'sentence2'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
test

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_dev_00000,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
1,klue-sts-v1_dev_00001,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,"{'label': 1.4, 'real-label': 1.428571428571429...","{'agreement': '0:4:3:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_dev_00002,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,"{'label': 1.3, 'real-label': 1.285714285714286...","{'agreement': '0:5:2:0:0:0', 'annotators': ['0..."
3,klue-sts-v1_dev_00003,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.","{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['1..."
4,klue-sts-v1_dev_00004,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,"{'label': 2.5, 'real-label': 2.5, 'binary-labe...","{'agreement': '1:0:1:3:1:0', 'annotators': ['0..."
...,...,...,...,...,...,...
514,klue-sts-v1_dev_00514,policy-rtt,"문체부는 이를 연차적으로 확대, 시행해 학교운동부와 스포츠클럽 간의 연계를 강화한다.",문화체육관광부는 학교스포츠학과와 스포츠클럽의 연계성을 강화하기 위해 매년 이 프로그...,"{'label': 2.2, 'real-label': 2.2, 'binary-labe...","{'agreement': '0:1:2:2:0:0', 'annotators': ['0..."
515,klue-sts-v1_dev_00515,airbnb-sampled,일단 정확한 정보와 빠른 답변이 정말 좋았습니다.,호스트의 빠른 답변과 유용한 정보들이 정말 좋습니다.,"{'label': 2.8, 'real-label': 2.833333333333333...","{'agreement': '0:0:1:5:0:0', 'annotators': ['0..."
516,klue-sts-v1_dev_00516,airbnb-sampled,게스트에 대한 배려가 묻어나는 시설들이었습니다.,우선 공간에 대한 센스가 돋보이는 곳이었습니다.,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
517,klue-sts-v1_dev_00517,policy-sampled,밤하늘을 배경으로 ‘비대면 드론쇼’도 펼쳐진다.,‘비대면 실감형 문화공연 플랫폼’ 개념도.,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '5:0:1:0:0:0', 'annotators': ['0..."


In [48]:
test.duplicated(['sentence1', 'sentence2']).sum()

0

In [49]:
test.isnull().sum()

guid           0
source         0
sentence1      0
sentence2      0
labels         0
annotations    0
dtype: int64

In [50]:
# 라벨 분리
labels2 = test.labels.to_list()
labels2 = pd.DataFrame(labels2)
print(len(labels2))
labels2

519


,label,real-label,binary-label
0,4.9,4.857143,1
1,1.4,1.428571,0
2,1.3,1.285714,0
3,3.7,3.714286,1
4,2.5,2.500000,0
...,...,...,...
514,2.2,2.200000,0
515,2.8,2.833333,0
516,0.3,0.333333,0
517,0.3,0.333333,0


In [51]:
test = test[['sentence1', 'sentence2']].join(labels2['binary-label'])
test

,sentence1,sentence2,binary-label
0,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",1
1,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,0
2,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,0
3,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",1
4,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,0
...,...,...,...
514,"문체부는 이를 연차적으로 확대, 시행해 학교운동부와 스포츠클럽 간의 연계를 강화한다.",문화체육관광부는 학교스포츠학과와 스포츠클럽의 연계성을 강화하기 위해 매년 이 프로그...,0
515,일단 정확한 정보와 빠른 답변이 정말 좋았습니다.,호스트의 빠른 답변과 유용한 정보들이 정말 좋습니다.,0
516,게스트에 대한 배려가 묻어나는 시설들이었습니다.,우선 공간에 대한 센스가 돋보이는 곳이었습니다.,0
517,밤하늘을 배경으로 ‘비대면 드론쇼’도 펼쳐진다.,‘비대면 실감형 문화공연 플랫폼’ 개념도.,0


In [52]:
def set_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"# available GPUs : {torch.cuda.device_count()}")
        print(f"GPU name : {torch.cuda.get_device_name()}")
    else:
        device = torch.device("cpu")
    return device
device = set_device()
device

# available GPUs : 1
GPU name : Tesla T4


device(type='cuda')

In [53]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
from transformers import AutoTokenizer, AutoModel, RobertaModel, RobertaTokenizer
from transformers import get_linear_schedule_with_warmup, get_constant_schedule
from sklearn.model_selection import train_test_split

In [55]:
# seed
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [56]:
train, val = train_test_split(df, test_size=0.3, shuffle=True)

In [57]:
from functools import partial

In [58]:
from transformers import AutoTokenizer

In [59]:
class CustomDataset(Dataset):
    def __init__(self, sentence1, sentence2, real_label):
        self.X1 = sentence1 #list str
        self.X2 = sentence2 #list str
        self.Y = real_label #list float

    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, index):
        return self.X1[index], self.X2[index], self.Y[index]

In [60]:
def generate_dataset(df, flag):
    sen_one = df['sentence1'].tolist()
    sen_two = df['sentence2'].tolist()
    real_lab = df['binary-label'].tolist()

    return CustomDataset(sen_one, sen_two, real_lab)

In [61]:
train_dataset = generate_dataset(train, True)
val_dataset = generate_dataset(val, True)
test_dataset = generate_dataset(test, True)

In [62]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [63]:
def CustomCollateFn(batch):
    sen_one_list = []
    sen_two_list = []
    label_list = []


    for sen_one, sen_two, label in batch:
        sen_one_list.append(sen_one)
        sen_two_list.append(sen_two)
        label_list.append(label)
    
    tokenized_sen_one = tokenizer(sen_one_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=128, return_tensors='pt')
    tokenized_sen_two = tokenizer(sen_two_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=128, return_tensors='pt')

    label_list = torch.Tensor(label_list)


    return (tokenized_sen_one, tokenized_sen_two, label_list)

In [64]:
def CustomCollateFn_dev(batch):
    sen_one_list = []
    sen_two_list = []
    label_list = []


    for sen_one, sen_two, label in batch:
        sen_one_list.append(sen_one)
        sen_two_list.append(sen_two)
        label_list.append(label)

    tokenized_sen_one = tokenizer(sen_one_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=128, return_tensors='pt')
    tokenized_sen_two = tokenizer(sen_two_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=128, return_tensors='pt')

    label_list = torch.Tensor(label_list)

    return (tokenized_sen_one, tokenized_sen_two, label_list)

In [65]:
def mean_pooling_fn(output, attention_mask):
    embedding = output.last_hidden_state # (batch len, longest sentence length, 1024)
    att_msk = attention_mask # (batch_len, 1024)
    mask = att_msk.unsqueeze(-1).expand(output.last_hidden_state.size()).float() # (batch len, longest sentence length, 1024)
    masked_embedding = output.last_hidden_state * mask # (batch_len, longest sen len, 1024)
    me_sum = torch.sum(masked_embedding, 1) # (batch_len, 1024)
    ms_sum = torch.clamp(mask.sum(1), min=1e-9) # (batch_len, 1024)
    mean_pool = me_sum/ms_sum # batch_len, 1024
    return mean_pool

In [66]:
class CustomPooling(nn.Module):
    def __init__(self):
        super(CustomPooling, self).__init__()

        self.robert = AutoModel.from_pretrained("klue/roberta-large")


        self.cos_score = nn.Sequential(
            nn.Identity()
        )
    
    def forward(self, senone, sentwo):
        output_one = self.robert(input_ids=senone['input_ids'], attention_mask=senone['attention_mask'],
                             token_type_ids=senone['token_type_ids'])
        output_two = self.robert(input_ids=sentwo['input_ids'], attention_mask=sentwo['attention_mask'],
                             token_type_ids=sentwo['token_type_ids'])

        pooled_one = mean_pooling_fn(output_one, senone['attention_mask'])
        pooled_two = mean_pooling_fn(output_two, sentwo['attention_mask'])


        cos_sim = torch.cosine_similarity(pooled_one, pooled_two)
        logit = self.cos_score(cos_sim)

        return logit

In [67]:
def initializer(input_dataloader, epochs):
    model = CustomPooling()
    optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
    print(f'total step: {len(input_dataloader) * epochs}')

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps = round(len(input_dataloader)*0.1),
        num_training_steps = len(input_dataloader) * epochs,

    )

    return model, optimizer, scheduler

In [68]:
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss):
    file_name = f'/content/drive/MyDrive/data/checkpoints/sts_vx.ckpt.{epoch}'
    torch.save({
        'epoch':epoch,
        'model_state_dict':model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'scheduler_state_dict':scheduler.state_dict(),
        'loss':loss
    }, file_name)

    print(f'SAVING EPOCH {epoch} ...')

In [69]:
def train(model, loss_fct, scheduler, optimizer, train_dataloader, valid_dataloader, epochs):
    for epoch in range(epochs):
        print(f'****** STARTING TO TRAIN EPOCH #{epoch} ******')

        total_loss = 0
        batch_loss = 0
        batch_count = 0

        model.train()
        model.to(device)

        for step, batch in enumerate(train_dataloader):
            batch_count += 1
            batch = tuple(items.to(device) for items in batch)

            (x_batch_one, x_batch_two, y_batch) = batch

            model.zero_grad()

            logit = model(x_batch_one, x_batch_two)
            loss = loss_fct(logit, y_batch)

            batch_loss += loss.item()
            total_loss += loss.item()

            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            if(step % 10 == 0 and step != 0):

                writer.add_scalar(
                    tag = "Train Loss",
                    scalar_value = batch_loss / batch_count,
                    global_step = epoch * len(train_dataloader) + step
                )

                # 학습 learning rate 기록
                writer.add_scalar(
                    tag = "Train LR",
                    scalar_value = optimizer.param_groups[0]['lr'],
                    global_step = epoch * len(train_dataloader) + step
                )

                learning_rate = optimizer.param_groups[0]['lr']
                print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate}, Avg Loss : {batch_loss / batch_count:.4f}")

                batch_loss, batch_count = 0,0

        print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
        print(f"*****Epoch {epoch} Train Finish*****\n")

        if valid_dataloader is not None:
            print(f"*****Epoch {epoch} Valid Start*****")
            valid_loss, valid_pearson, valid_f1 = validate(model, loss_fct, valid_dataloader)
            print(f"Epoch {epoch} Valid Loss : {valid_loss} Valid Pearsonr : {valid_pearson} ValidF1 : {valid_f1}")
            
            writer.add_scalar(
                tag = "Valid Loss",
                scalar_value = valid_loss,
                global_step = epoch
            )

            writer.add_scalar(
                tag = "Valid Pearsonr",
                scalar_value = valid_pearson,
                global_step = epoch
            )

            writer.add_scalar(
                tag = "Valid F1",
                scalar_value = valid_f1,
                global_step = epoch
            )
            
            print(f"*****Epoch {epoch} Valid Finish*****\n")
        
        save_checkpoint(".", model, optimizer, scheduler, epoch, total_loss/(step+1))
        torch.cuda.empty_cache()
    print('** Train Completed! **')

In [70]:
!pip install audtorch
from audtorch.metrics.functional import pearsonr
from sklearn.metrics import f1_score
from scipy import stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from torch.utils.tensorboard import SummaryWriter
!pip install jupyter-tensorboard

writer = SummaryWriter('/content/drive/MyDrive/data/sts_logs/vx')
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
%tensorboard --logdir {"/content/drive/MyDrive/data/sts_logs/v2"}

Reusing TensorBoard on port 6006 (pid 593), started 5:23:10 ago. (Use '!kill 593' to kill it.)

<IPython.core.display.Javascript object>

In [73]:
def validate(model, loss_fct, valid_dataloader):

    model.eval()
    model.to(device)

    total_loss = 0
    total_acc = 0
    all_prediction = []
    all_reallabel = []

    for step, batch in enumerate(valid_dataloader):
        batch = tuple(items.to(device) for items in batch)

        (x_batch_one, x_batch_two, batch_y) = batch

        with torch.no_grad():
            logit = model(x_batch_one, x_batch_two)


        loss = loss_fct(logit, batch_y)
        total_loss += loss.item()


        logit = logit[:].cpu()
        batch_y = batch_y.cpu()

        print(f'Step: {step},  Pearson: {pearsonr(logit, batch_y)}')

        all_prediction = all_prediction + logit.tolist()
        all_reallabel = all_reallabel + batch_y.tolist()

    #pearson

    pred = torch.Tensor(all_prediction) # x
    real = torch.Tensor(all_reallabel) # y
    
    pearson = pearsonr(pred, real)
    
    #loss
    total_loss = total_loss / (step+1)

    #f1
    fone = f1_process(pred, real)

    return total_loss, pearson, fone

In [85]:
def f1_process(pred, real):
    bin_real = []
    bin_pred = []

    for index in range(len(real)):
        if real[index] < 1:
            bin_real.append(0)
        else:
            bin_real.append(1)
        if pred[index] < 1:
            bin_pred.append(0)
        else:
            bin_pred.append(1)
    return f1_score(bin_real, bin_pred)

In [75]:
device = torch.device("cuda")
train_dataloader = DataLoader(
    train_dataset,
    batch_size = 16,
    sampler = RandomSampler(train_dataset),
    collate_fn = CustomCollateFn,
)
valid_dataloader = DataLoader(
    val_dataset,
    batch_size = 32,
    sampler = SequentialSampler(val_dataset),
    collate_fn = CustomCollateFn_dev,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size = 32,
    sampler = SequentialSampler(test_dataset),
    collate_fn = CustomCollateFn_dev,
) 

In [76]:
epochs = 4
loss_fct = MSELoss()
model, optimizer, scheduler = initializer(train_dataloader, epochs)
train(model, loss_fct, scheduler, optimizer, train_dataloader, valid_dataloader, epochs)

# validate(model, loss_fct, valid_dataloader, device)
writer.close()

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

total step: 4244
****** STARTING TO TRAIN EPOCH #0 ******
Epoch: 0, Step : 10, LR : 1.037735849056604e-06, Avg Loss : 0.3538
Epoch: 0, Step : 20, LR : 1.981132075471698e-06, Avg Loss : 0.3619
Epoch: 0, Step : 30, LR : 2.9245283018867924e-06, Avg Loss : 0.3339
Epoch: 0, Step : 40, LR : 3.8679245283018875e-06, Avg Loss : 0.2661
Epoch: 0, Step : 50, LR : 4.811320754716982e-06, Avg Loss : 0.2137
Epoch: 0, Step : 60, LR : 5.754716981132076e-06, Avg Loss : 0.1707
Epoch: 0, Step : 70, LR : 6.69811320754717e-06, Avg Loss : 0.1384
Epoch: 0, Step : 80, LR : 7.641509433962266e-06, Avg Loss : 0.1279
Epoch: 0, Step : 90, LR : 8.58490566037736e-06, Avg Loss : 0.1433
Epoch: 0, Step : 100, LR : 9.528301886792455e-06, Avg Loss : 0.1239
Epoch: 0, Step : 110, LR : 9.9879168680522e-06, Avg Loss : 0.1214
Epoch: 0, Step : 120, LR : 9.963750604156598e-06, Avg Loss : 0.1220
Epoch: 0, Step : 130, LR : 9.939584340260995e-06, Avg Loss : 0.1113
Epoch: 0, Step : 140, LR : 9.915418076365395e-06, Avg Loss : 0.1023
E

In [84]:
def predict(model, test_dataloader):
    model.eval()
    model.to(device)

    all_prediction = []
    all_reallabel = []

    for step, batch in enumerate(test_dataloader):

        batch = tuple(items.to(device) for items in batch)

        (x_batch_one, x_batch_two, batch_y) = batch

        with torch.no_grad():
            logit = model(x_batch_one, x_batch_two)
        logit = logit*5

        logit = logit.cpu()
        batch_y = batch_y.cpu()

        all_prediction = all_prediction + logit.tolist()
        all_reallabel = all_reallabel + batch_y.tolist()

    pred = torch.Tensor(all_prediction) # x
    real = torch.Tensor(all_reallabel) # y
    
    pearson = pearsonr(pred, real) #stats.spearmanr(pred, real)

    #f1
    fone = f1_process(pred, real)

    return pearson, fone

In [80]:
ckpt1 = '/content/drive/MyDrive/data/checkpoints/sts_vx.ckpt.0'
ckpt2 = '/content/drive/MyDrive/data/checkpoints/sts_vx.ckpt.1'
ckpt3 = '/content/drive/MyDrive/data/checkpoints/sts_vx.ckpt.2'
ckpt4 = '/content/drive/MyDrive/data/checkpoints/sts_vx.ckpt.3'

In [86]:
all_checkpoints = [ckpt1, ckpt2, ckpt3, ckpt4]

for checkpoint in all_checkpoints:
    loaded_ckpt = torch.load(checkpoint)
    model, optimizer, scheduler = initializer(train_dataloader, 1)
    model.load_state_dict(loaded_ckpt['model_state_dict'])
    pearson_score, fonescore = predict(model, test_dataloader)
    print(f'{checkpoint[44:]} pearsonr: {pearson_score}, f1_score: {fonescore}')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

total step: 1061
vx.ckpt.0 pearsonr: tensor([0.7127]), f1_score: 0.7073954983922829


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

total step: 1061
vx.ckpt.1 pearsonr: tensor([0.7115]), f1_score: 0.7236842105263158


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

total step: 1061
vx.ckpt.2 pearsonr: tensor([0.7065]), f1_score: 0.7296849087893864


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

total step: 1061
vx.ckpt.3 pearsonr: tensor([0.7075]), f1_score: 0.7345575959933222
